In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from torchvision.transforms import functional as F
from scipy.ndimage import rotate
import time
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from tqdm.auto import tqdm
from torchvision.transforms import v2

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset
class CSVDataset(Dataset):
    def __init__(self, csv_file):
        # Load the data
        self.data = pd.read_csv(csv_file)

    def __getitem__(self, index):
        # Get the data for one example
        example_data = self.data.iloc[index]
        # Separate the features from the target
        features = torch.tensor(example_data[2:].values, dtype=torch.float32).reshape(1, 28, 28)
        target = torch.tensor(example_data.iloc[1], dtype=torch.long)
        return features, target

    def __len__(self):
        return len(self.data)
    
    def classes(self, index):
        return self.data.iloc[index].iloc[1]


# Create instances of the dataset
train_data = CSVDataset('final_dataset/train_dataset.csv')
test_data = CSVDataset('final_dataset/test_dataset.csv')

In [5]:
image, label = train_data[1134]
label.item(), image.shape

(9, torch.Size([1, 28, 28]))

In [6]:
classes_list = set()
for i in range(len(test_data)):
    _, target = test_data[i]
    classes_list.add(target.item())

classes_list

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61}

In [7]:
len(classes_list)

62

In [8]:
BATCH_SIZE = 16

train_dataloader = DataLoader(train_data, #Dataset to iterate on for the according batch size.
                              batch_size = BATCH_SIZE, #Size of every single iteration.
                              shuffle = True, #To make the loaded data selected randomly.
                              )
test_dataloader = DataLoader(test_data,
                             batch_size = BATCH_SIZE,
                             shuffle = True)

print(f"For training data there is {len(train_dataloader)} iterations with the batch size of {BATCH_SIZE}.")
print(f"For testing data there is {len(test_dataloader)} iterations with the batch size of {BATCH_SIZE}.")

For training data there is 23057 iterations with the batch size of 16.
For testing data there is 3901 iterations with the batch size of 16.


In [13]:
import torch.nn as nn
import torch.nn.functional as F

class LetterRecognizerModel1(nn.Module):
    def __init__(self, input_shape, hidden_units, output_size):
        super().__init__()
        self.conv1 = nn.Conv2d(input_shape, hidden_units, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_units)
        self.conv2 = nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(hidden_units)
        self.conv3 = nn.Conv2d(hidden_units, hidden_units, kernel_size=5, stride=2, padding=2)
        self.bn3 = nn.BatchNorm2d(hidden_units)
        self.dropout1 = nn.Dropout(0.4)

        self.conv4 = nn.Conv2d(hidden_units, hidden_units*2, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(hidden_units*2)
        self.conv5 = nn.Conv2d(hidden_units*2, hidden_units*2, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(hidden_units*2)
        self.conv6 = nn.Conv2d(hidden_units*2, hidden_units*2, kernel_size=5, stride=2, padding=2)
        self.bn6 = nn.BatchNorm2d(hidden_units*2)
        self.dropout2 = nn.Dropout(0.4)

        self.conv7 = nn.Conv2d(hidden_units*2, hidden_units*4, kernel_size=4)
        self.bn7 = nn.BatchNorm2d(hidden_units*4)
        self.dropout3 = nn.Dropout(0.4)
        self.fc = nn.Linear(hidden_units*4, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout1(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.dropout2(x)

        x = F.relu(self.bn7(self.conv7(x)))
        x = self.dropout3(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.softmax(self.fc(x), dim=1)
        return x
torch.manual_seed(42)
model_1 = LetterRecognizerModel1(input_shape=1,
    hidden_units=10,
    output_size=len(classes_list)).to(device)
model_1

LetterRecognizerModel1(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(10, 10, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bn3): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (conv4): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(20, 20, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bn6): BatchNorm2d(20, e